该notebook中的模型，可以做到与biogeme的结果基本一致，包括对于模型的Structural equation，Measurement equation以及Utility function中的参数的估计，对应研究进展20181023。

# Data and Package Prepare

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.constraints import min_max_norm

In [2]:
train_data = pd.read_csv('optif.csv', low_memory=False)
#train_data = pd.read_csv('D:/ProFile/Latent/biogeme/optima_keras.csv', low_memory=False)
random_num = 500
# S=tf.Session()

In [3]:
trainable = True

In [4]:
train_latent = train_data[['ContIncome_0_4000','ContIncome_10000_more', 'ContIncome_4000_6000', 
                           'ContIncome_6000_8000', 'ContIncome_8000_10000', 
                           'age_65_more','haveChildren', 
                           'haveGA', 'highEducation', 'individualHouse', 'male',
                           'moreThanOneBike', 'moreThanOneCar']]

train_PTcom = train_data[['CostPT_HWH', 'CostPT_Other']]
train_CARcom = train_data[['CostCar_HWH', 'CostCar_Other']]

train_PTasc = train_data[['TimePT', 'WaitingTimePT']]
train_CARasc = train_data[['TimeCar']]
train_SMasc = train_data[['distance_km']]

train_indicator = train_data[['Envir01','Envir02','Envir03','Mobil11','Mobil14','Mobil16','Mobil17']]

train_y = train_data[['Choice']]

# Latent Variable Submodel

In [5]:
class LL(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(LL, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = (train_indicator.shape[1], )
        # Create a trainable weight variable for this layer.
        self.sigma = self.add_weight(name='sigma',
                                      shape=shape,
                                     initializer=keras.initializers.Constant(value=np.ones(train_indicator.shape[1])),
#                                      initializer=keras.initializers.Constant(
#                                         value=np.array([1.25,1.12,1.07,1.08,1.05,1.1,1.11])),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(LL, self).build(input_shape)

    def call(self, inputs):
#         error = tf.subtract(inputs[0],inputs[1])
#         se=K.square(error)
        se=K.square(inputs)
        term1 = tf.divide(se, tf.multiply(2., K.square(self.sigma)))
        term2 = K.log(self.sigma)
        term3 = tf.multiply(0.5, K.log(6.283185))
        return tf.add(term1, tf.add(term2, term3))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(LL, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [6]:
class Subtract(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Subtract, self).__init__(**kwargs)

    def build(self, input_shape):

        # Be sure to call this at the end
        super(Subtract, self).build(input_shape)

    def call(self, inputs):
        return tf.cast(tf.subtract(inputs[0],inputs[1]),tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        base_config = super(Subtract, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [7]:
def LL_loss(y_true, y_pred):
    return K.sum(K.sum(K.abs(y_pred - y_true), axis=0))

In [8]:
def multi_indicator_loss(y_true, y_pred):
    n=train_data.shape[0]
    k1,k2 = 14,16
    error2 = K.sum(K.abs(y_pred - y_true), axis=0)
    term1 = tf.multiply(0.5, n-k2)
    term2_1 = tf.multiply(0.5, tf.log(tf.divide(error2, n-k2)))
    term2_2 = tf.multiply(0.5, tf.log(6.283185))
    term2 = tf.multiply(tf.cast(n, tf.float32), tf.add(term2_1, term2_2))
    L2 = tf.add(term1, term2)

    return K.sum(L2)

In [9]:
input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
y_ture = keras.Input(shape = (train_indicator.shape[1],), name = 'input_y_ture')
# car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear',
#                             kernel_initializer=keras.initializers.Constant(
#                                 value=np.array([[ -0.0265 ],
#                                                 [-0.0863],
#                                                 [0.2],
#                                                 [-0.205],
#                                                 [0.541],
#                                                 [-0.077],
#                                                 [0.0182],
#                                                 [ 0.724],
#                                                 [ 0.288],
#                                                 [ 0.0764],
#                                                 [-0.0694],
#                                                 [ 0.293],
#                                                 [-0.68]])),
#                                bias_initializer=keras.initializers.Constant(value=2.27))(input_latent)
hidden = keras.layers.Dense(4, name = 'hidden', activation = 'relu')(input_latent)
car_sys = keras.layers.Dense(1, name = 'car_sys', activation = 'linear')(hidden)

indicator_ref = keras.layers.Dense(1, name = 'indicator_ref',activation = 'linear',
                               kernel_initializer=keras.initializers.Constant(value=1),
                               bias_initializer=keras.initializers.Constant(value=0))(car_sys)
indicator_other = keras.layers.Dense(train_indicator.shape[1]-1, 
                                     name = 'indicator_other',
                                   kernel_initializer=keras.initializers.RandomUniform(minval=-1., maxval=0., seed=None),
#                                      kernel_initializer=keras.initializers.Constant(
#                                          value=np.array([0.493,-0.706,-0.626,-0.707,-0.542,-0.456])),
#                                      bias_initializer=keras.initializers.Constant(
#                                          value=np.array([2.02,4.64,5.3,4.9,4.79,4.53,])),
                                     activation = 'linear')(car_sys)

indicator = keras.layers.Concatenate(axis=-1)([indicator_ref,indicator_other])

error = Subtract(1)([indicator, y_ture])

loglikelihood = LL(1)(error)

# idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
#                     outputs = indicator)

idmodel=tf.keras.Model(inputs = [input_latent, y_ture],
                    outputs = loglikelihood)

#idmodel.layers[3].trainable = False
idmodel.layers[2].trainable = False
#idmodel.layers[7].trainable = False

idmodel.compile(optimizer=keras.optimizers.Adam(0.01),
              loss = LL_loss)

LLoutput = np.zeros((train_latent.shape[0],train_indicator.shape[1]))

In [10]:
idmodel.evaluate([train_latent, train_indicator], 
          LLoutput, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 3s 2ms/step


100307.703125

In [12]:
idmodel.fit([train_latent, train_indicator], 
         LLoutput, epochs=8000, batch_size = train_latent.shape[0])

Epoch 1/8000
1545/1545 [==============================] - 0s 185us/step - loss: 100307.7188
Epoch 2/8000
1545/1545 [==============================] - 0s 4us/step - loss: 95151.5156
Epoch 3/8000
1545/1545 [==============================] - 0s 4us/step - loss: 89790.6016
Epoch 4/8000
1545/1545 [==============================] - 0s 4us/step - loss: 84789.2500
Epoch 5/8000
1545/1545 [==============================] - 0s 3us/step - loss: 80311.4688
Epoch 6/8000
1545/1545 [==============================] - 0s 3us/step - loss: 76412.9609
Epoch 7/8000
1545/1545 [==============================] - 0s 6us/step - loss: 73066.2891
Epoch 8/8000
1545/1545 [==============================] - 0s 4us/step - loss: 70176.2969
Epoch 9/8000
1545/1545 [==============================] - 0s 5us/step - loss: 67703.0859
Epoch 10/8000
1545/1545 [==============================] - 0s 4us/step - loss: 65560.7266
Epoch 11/8000
1545/1545 [==============================] - 0s 4us/step - loss: 63628.9805
Epoch 12/8000
15

1545/1545 [==============================] - 0s 4us/step - loss: 20725.8301
Epoch 93/8000
1545/1545 [==============================] - 0s 3us/step - loss: 20650.0430
Epoch 94/8000
1545/1545 [==============================] - 0s 3us/step - loss: 20577.0977
Epoch 95/8000
1545/1545 [==============================] - 0s 4us/step - loss: 20506.8359
Epoch 96/8000
1545/1545 [==============================] - 0s 4us/step - loss: 20439.1270
Epoch 97/8000
1545/1545 [==============================] - 0s 3us/step - loss: 20373.8496
Epoch 98/8000
1545/1545 [==============================] - 0s 3us/step - loss: 20310.8887
Epoch 99/8000
1545/1545 [==============================] - 0s 4us/step - loss: 20250.1250
Epoch 100/8000
1545/1545 [==============================] - 0s 3us/step - loss: 20191.4414
Epoch 101/8000
1545/1545 [==============================] - 0s 4us/step - loss: 20134.7383
Epoch 102/8000
1545/1545 [==============================] - 0s 3us/step - loss: 20079.9277
Epoch 103/8000
1545/1

Epoch 183/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18354.8203
Epoch 184/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18347.8125
Epoch 185/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18340.9434
Epoch 186/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18334.2109
Epoch 187/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18327.6152
Epoch 188/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18321.1484
Epoch 189/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18314.8047
Epoch 190/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18308.5859
Epoch 191/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18302.4883
Epoch 192/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18296.5117
Epoch 193/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18290.6406

Epoch 274/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18040.5371
Epoch 275/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18038.9258
Epoch 276/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18037.3281
Epoch 277/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18035.7461
Epoch 278/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18034.1797
Epoch 279/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18032.6309
Epoch 280/8000
1545/1545 [==============================] - 0s 4us/step - loss: 18031.0977
Epoch 281/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18029.5781
Epoch 282/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18028.0762
Epoch 283/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18026.5859
Epoch 284/8000
1545/1545 [==============================] - 0s 3us/step - loss: 18025.1016

1545/1545 [==============================] - 0s 4us/step - loss: 17935.8613
Epoch 365/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17934.9746
Epoch 366/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17934.0918
Epoch 367/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17933.2129
Epoch 368/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17932.3379
Epoch 369/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17931.4688
Epoch 370/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17930.5996
Epoch 371/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17929.7383
Epoch 372/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17928.8789
Epoch 373/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17928.0254
Epoch 374/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17927.1738
Epoch 375/8000

...

1545/1545 [==============================] - 0s 3us/step - loss: 17394.3203
Epoch 7642/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.4336
Epoch 7643/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.5098
Epoch 7644/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.6094
Epoch 7645/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.6992
Epoch 7646/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.6895
Epoch 7647/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.6680
Epoch 7648/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.5273
Epoch 7649/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.4258
Epoch 7650/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.2617
Epoch 7651/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.2734
Epoc

Epoch 7731/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.1934
Epoch 7732/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.2031
Epoch 7733/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1758
Epoch 7734/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.1699
Epoch 7735/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1895
Epoch 7736/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1875
Epoch 7737/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1914
Epoch 7738/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1797
Epoch 7739/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.1602
Epoch 7740/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.2031
Epoch 7741/8000
1545/1545 [==============================] - 0s 4us/step - loss:

1545/1545 [==============================] - 0s 3us/step - loss: 17394.2520
Epoch 7821/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.2227
Epoch 7822/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.2109
Epoch 7823/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.1504
Epoch 7824/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1016
Epoch 7825/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1133
Epoch 7826/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1074
Epoch 7827/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.1074
Epoch 7828/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.0820
Epoch 7829/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.0859
Epoch 7830/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.0664
Epoc

Epoch 7910/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.2012
Epoch 7911/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.2832
Epoch 7912/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.3828
Epoch 7913/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.3945
Epoch 7914/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.5273
Epoch 7915/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.5273
Epoch 7916/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.5332
Epoch 7917/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.5078
Epoch 7918/8000
1545/1545 [==============================] - 0s 3us/step - loss: 17394.4160
Epoch 7919/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17394.2930
Epoch 7920/8000
1545/1545 [==============================] - 0s 3us/step - loss:

Epoch 8000/8000
1545/1545 [==============================] - 0s 4us/step - loss: 17392.7578


In [13]:
i = 0
for layer in idmodel.layers:
    print(layer.name, i)
    i += 1

input_latent 0
hidden 1
car_sys 2
indicator_ref 3
indicator_other 4
concatenate 5
input_y_ture 6
subtract 7
ll 8


In [15]:
idmodel.layers[8].get_weights()

[array([1.3518063, 1.1721292, 1.2000321, 1.1279353, 1.1813884, 1.1423569,
        1.2997715], dtype=float32)]

# Discrete Choice Submodel

计算效用中公共系数部分

V0 alternative specific part

In [16]:
# input [carlover, time_PT_scaled, waitingtimePT]
class V0_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V0_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_time_PT = self.add_weight(name='beta_time_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.72),
                                      # constraints = min_max_norm(min_value=-1.020001, max_value=-1.019999, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_cl_PT = self.add_weight(name='beta_cl_PT',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=1.37),
                                      # constraints = min_max_norm(min_value=1.929999, max_value=1.930001, rate=1.0, axis=0),
                                      trainable=trainable)
        self.beta_waiting_time = self.add_weight(name='self.beta_waiting_time',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-0.0275),
                                      # constraints = min_max_norm(min_value=-0.075001, max_value=-0.074999, rate=1.0, axis=0),           
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V0_as, self).build(input_shape)

    def call(self, inputs):
        cl = tf.multiply(inputs[0], self.beta_cl_PT)
        timePT = tf.transpose(tf.multiply(self.beta_time_PT, inputs[1][:,0]))
        waitingtime = tf.transpose(tf.multiply(inputs[1][:,1], self.beta_waiting_time))
        # return tf.transpose(tf.add(timePT, waitingtime))
        return tf.add(timePT, tf.add(waitingtime, cl))
        # return [timePT, waitingtime]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V0_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V1 alternative specific part

In [17]:
# input [carlover, time_CAR_scaled]
class V1_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V1_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.asc_CAR = self.add_weight(name='asc_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=4.66),
                                      # constraints = min_max_norm(min_value=0.740999, max_value=0.741001, rate=1.0, axis=0),
                                      trainable=trainable)
        
        self.beta_time_CAR = self.add_weight(name='beta_time_CAR',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-5.83),
                                      # constraints = min_max_norm(min_value=0.482999, max_value=0.483001, rate=1.0, axis=0),          
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V1_as, self).build(input_shape)

    def call(self, inputs):
        # cl = tf.multiply(inputs[0], self.beta_cl_CAR)
        
        timeCAR = tf.multiply(inputs[1], self.beta_time_CAR)
        
        # return tf.transpose(tf.add(timeCAR, self.asc_CAR))
        # return tf.add(tf.add(cl,timeCAR), self.asc_CAR)
        return tf.add(timeCAR, self.asc_CAR)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V1_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

V2 alternative specific part

In [18]:
class V2_as(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(V2_as, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Create a trainable weight variable for this layer.
        self.beta_dist = self.add_weight(name='beta_dist',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.22),
                                      # constraints = min_max_norm(min_value=-4.88001, max_value=-4.87999, rate=1.0, axis=0),   
                                      trainable=trainable)
        self.asc_SM = self.add_weight(name='asc_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=-1.62),
                                      # constraints = min_max_norm(min_value=1.23999, max_value=1.24001, rate=1.0, axis=0), 
                                      trainable=trainable)
        self.beta_cl_SM = self.add_weight(name='beta_cl_SM',
                                      shape=shape,
                                      initializer=keras.initializers.Constant(value=2.),
                                      # constraints = min_max_norm(min_value=-20.4001, max_value=-20.3999, rate=1.0, axis=0),
                                      trainable=trainable)
        # Be sure to call this at the end
        super(V2_as, self).build(input_shape)

    def call(self, inputs):
        cl_SM = tf.multiply(inputs[0], self.beta_cl_SM)
        dist = tf.multiply(inputs[1], self.beta_dist)
        
        return tf.add(tf.add(cl_SM, dist), self.asc_SM)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(V2_as, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [19]:
class TransToRandShape(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(TransToRandShape, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(TransToRandShape, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.zeros((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(TransToRandShape, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [20]:
class Add_Random(keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Add_Random, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((1, self.output_dim))
        # Be sure to call this at the end
        super(Add_Random, self).build(input_shape)

    def call(self, inputs):
               
        return tf.add(inputs,tf.random_normal((tf.shape(inputs)[0],random_num)))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


    def get_config(self):
        base_config = super(Add_Random, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)

Model Specification

In [22]:
### input layers
# batch_size = 32

input_latent = keras.Input(shape = (train_latent.shape[1],), name = 'input_latent')
input_PTcom = keras.Input(shape = (train_PTcom.shape[1],), name = 'input_PTcom')
input_CARcom = keras.Input(shape = (train_CARcom.shape[1],), name = 'input_CARcom')
input_PTasc = keras.Input(shape = (train_PTasc.shape[1],), name = 'input_PTasc')
input_CARasc = keras.Input(shape = (train_CARasc.shape[1],), name = 'input_CARasc')
input_SMasc = keras.Input(shape = (train_SMasc.shape[1],), name = 'input_SMasc')

### car lover
idmodel.layers[1].trainable = False
hidden = idmodel.layers[1](input_latent)
car_sys = idmodel.layers[2](hidden)
# carlover = keras.layers.Lambda(lambda x: x + tf.random_normal((1505,random_num)), name = 'carlover')(car_sys)
carlover = Add_Random(1,name = 'carlover')(car_sys)

### common coefficients part in utility
utilcommon = keras.layers.Dense(1, name = 'util_common', use_bias=False)
PTcom = TransToRandShape(1)(utilcommon(input_PTcom))
CARcom = TransToRandShape(1)(utilcommon(input_CARcom))


### alternative specific part in utility
# V0
# util_input0 = keras.layers.concatenate([carlover, input_PTasc], axis = -1)
util_V0as = V0_as(1)([carlover, input_PTasc])
V0 = keras.layers.Add(name = 'V0')([util_V0as, PTcom])

# V1
# util_input1 = keras.layers.concatenate([carlover, input_CARasc], axis = -1)
util_V1as = V1_as(1)([carlover, input_CARasc])
V1 = keras.layers.Add(name = 'V1')([util_V1as, CARcom])

# V2
_V2 = V2_as(1)([carlover, input_SMasc])
V2 =keras.layers.Lambda(lambda x: x + tf.zeros((1,random_num)), name = 'V2')(_V2)
### merge
# merged_vector=keras.layers.concatenate([[V0], [V1], [V2]], axis=-1)

prediction =keras.layers.Lambda(lambda x: tf.transpose(tf.reduce_mean(tf.nn.softmax(x, axis = 0),axis=2)), 
                                name = 'prediction')([V0, V1, V2])

model=tf.keras.Model(inputs = [input_latent, input_PTcom, input_CARcom, input_PTasc, input_CARasc, input_SMasc],
                    outputs = prediction)


model.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)


In [23]:
model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0])

1545/1545 [==============================] - 0s 133us/step


[14.424798011779785, 0.06796116381883621]

In [24]:
# tf.keras.backend.get_value(util_V1as)
# callbacks = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, epochs=4000, batch_size = train_latent.shape[0])

Epoch 1/4000
1545/1545 [==============================] - 0s 274us/step - loss: 14.4235 - acc: 0.0686
Epoch 2/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.4188 - acc: 0.0673
Epoch 3/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.4097 - acc: 0.0673
Epoch 4/4000
1545/1545 [==============================] - 0s 32us/step - loss: 14.4023 - acc: 0.0660
Epoch 5/4000
1545/1545 [==============================] - 0s 31us/step - loss: 14.3954 - acc: 0.0654
Epoch 6/4000
1545/1545 [==============================] - 0s 37us/step - loss: 14.3818 - acc: 0.0667
Epoch 7/4000
1545/1545 [==============================] - 0s 37us/step - loss: 14.3650 - acc: 0.0673
Epoch 8/4000
1545/1545 [==============================] - 0s 34us/step - loss: 14.3458 - acc: 0.0673
Epoch 9/4000
1545/1545 [==============================] - 0s 35us/step - loss: 14.3280 - acc: 0.0686
Epoch 10/4000
1545/1545 [==============================] - 0s 31us/step - loss: 14.3198 - 

Epoch 82/4000
1545/1545 [==============================] - 0s 41us/step - loss: 10.1478 - acc: 0.2893
Epoch 83/4000
1545/1545 [==============================] - 0s 39us/step - loss: 10.1344 - acc: 0.2913
Epoch 84/4000
1545/1545 [==============================] - 0s 38us/step - loss: 10.1209 - acc: 0.2919
Epoch 85/4000
1545/1545 [==============================] - 0s 37us/step - loss: 10.1102 - acc: 0.2906
Epoch 86/4000
1545/1545 [==============================] - 0s 37us/step - loss: 10.1036 - acc: 0.2919
Epoch 87/4000
1545/1545 [==============================] - 0s 32us/step - loss: 10.0955 - acc: 0.2932
Epoch 88/4000
1545/1545 [==============================] - 0s 32us/step - loss: 10.0887 - acc: 0.2945
Epoch 89/4000
1545/1545 [==============================] - 0s 37us/step - loss: 10.0814 - acc: 0.2945
Epoch 90/4000
1545/1545 [==============================] - 0s 35us/step - loss: 10.0749 - acc: 0.2984
Epoch 91/4000
1545/1545 [==============================] - 0s 34us/step - loss: 10

Epoch 163/4000
1545/1545 [==============================] - 0s 35us/step - loss: 8.3432 - acc: 0.3392
Epoch 164/4000
1545/1545 [==============================] - 0s 38us/step - loss: 8.2715 - acc: 0.3398
Epoch 165/4000
1545/1545 [==============================] - 0s 31us/step - loss: 8.1672 - acc: 0.3437
Epoch 166/4000
1545/1545 [==============================] - 0s 29us/step - loss: 8.0658 - acc: 0.3450
Epoch 167/4000
1545/1545 [==============================] - 0s 31us/step - loss: 7.9780 - acc: 0.3502
Epoch 168/4000
1545/1545 [==============================] - 0s 28us/step - loss: 7.8694 - acc: 0.3547
Epoch 169/4000
1545/1545 [==============================] - 0s 33us/step - loss: 7.7565 - acc: 0.3625
Epoch 170/4000
1545/1545 [==============================] - 0s 32us/step - loss: 7.6380 - acc: 0.3670
Epoch 171/4000
1545/1545 [==============================] - 0s 57us/step - loss: 7.5041 - acc: 0.3709
Epoch 172/4000
1545/1545 [==============================] - 0s 36us/step - loss: 7

Epoch 244/4000
1545/1545 [==============================] - 0s 34us/step - loss: 3.6182 - acc: 0.6667
Epoch 245/4000
1545/1545 [==============================] - 0s 38us/step - loss: 3.6032 - acc: 0.6660
Epoch 246/4000
1545/1545 [==============================] - 0s 35us/step - loss: 3.5921 - acc: 0.6680
Epoch 247/4000
1545/1545 [==============================] - 0s 38us/step - loss: 3.5785 - acc: 0.6686
Epoch 248/4000
1545/1545 [==============================] - 0s 39us/step - loss: 3.5569 - acc: 0.6706
Epoch 249/4000
1545/1545 [==============================] - 0s 38us/step - loss: 3.5537 - acc: 0.6718
Epoch 250/4000
1545/1545 [==============================] - 0s 37us/step - loss: 3.5372 - acc: 0.6731
Epoch 251/4000
1545/1545 [==============================] - 0s 35us/step - loss: 3.5178 - acc: 0.6738
Epoch 252/4000
1545/1545 [==============================] - 0s 40us/step - loss: 3.5158 - acc: 0.6757
Epoch 253/4000
1545/1545 [==============================] - 0s 33us/step - loss: 3

Epoch 325/4000
1545/1545 [==============================] - 0s 31us/step - loss: 2.7986 - acc: 0.7133
Epoch 326/4000
1545/1545 [==============================] - 0s 39us/step - loss: 2.7853 - acc: 0.7139
Epoch 327/4000
1545/1545 [==============================] - 0s 34us/step - loss: 2.7624 - acc: 0.7139
Epoch 328/4000
1545/1545 [==============================] - 0s 39us/step - loss: 2.7623 - acc: 0.7139
Epoch 329/4000
1545/1545 [==============================] - 0s 44us/step - loss: 2.7636 - acc: 0.7152
Epoch 330/4000
1545/1545 [==============================] - 0s 39us/step - loss: 2.7518 - acc: 0.7165
Epoch 331/4000
1545/1545 [==============================] - 0s 34us/step - loss: 2.7472 - acc: 0.7191
Epoch 332/4000
1545/1545 [==============================] - 0s 30us/step - loss: 2.7407 - acc: 0.7191
Epoch 333/4000
1545/1545 [==============================] - 0s 37us/step - loss: 2.7400 - acc: 0.7191
Epoch 334/4000
1545/1545 [==============================] - 0s 34us/step - loss: 2

...

Epoch 3613/4000
1545/1545 [==============================] - 0s 34us/step - loss: 0.6122 - acc: 0.7424
Epoch 3614/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6092 - acc: 0.7450
Epoch 3615/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6127 - acc: 0.7430
Epoch 3616/4000
1545/1545 [==============================] - 0s 28us/step - loss: 0.6104 - acc: 0.7405
Epoch 3617/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6122 - acc: 0.7437
Epoch 3618/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6121 - acc: 0.7430
Epoch 3619/4000
1545/1545 [==============================] - 0s 34us/step - loss: 0.6101 - acc: 0.7398
Epoch 3620/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6129 - acc: 0.7456
Epoch 3621/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6111 - acc: 0.7366
Epoch 3622/4000
1545/1545 [==============================] - 0s 30us/step

Epoch 3693/4000
1545/1545 [==============================] - 0s 45us/step - loss: 0.6100 - acc: 0.7411
Epoch 3694/4000
1545/1545 [==============================] - 0s 36us/step - loss: 0.6115 - acc: 0.7398
Epoch 3695/4000
1545/1545 [==============================] - 0s 33us/step - loss: 0.6117 - acc: 0.7398
Epoch 3696/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6102 - acc: 0.7424
Epoch 3697/4000
1545/1545 [==============================] - 0s 33us/step - loss: 0.6121 - acc: 0.7385
Epoch 3698/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6126 - acc: 0.7405
Epoch 3699/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6130 - acc: 0.7411
Epoch 3700/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6108 - acc: 0.7417
Epoch 3701/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6113 - acc: 0.7405
Epoch 3702/4000
1545/1545 [==============================] - 0s 29us/step

Epoch 3773/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6138 - acc: 0.7430
Epoch 3774/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6117 - acc: 0.7417
Epoch 3775/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6098 - acc: 0.7469
Epoch 3776/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6115 - acc: 0.7398
Epoch 3777/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6101 - acc: 0.7392
Epoch 3778/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6128 - acc: 0.7411
Epoch 3779/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6108 - acc: 0.7456
Epoch 3780/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6115 - acc: 0.7385
Epoch 3781/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6102 - acc: 0.7456
Epoch 3782/4000
1545/1545 [==============================] - 0s 31us/step

Epoch 3853/4000
1545/1545 [==============================] - 0s 33us/step - loss: 0.6117 - acc: 0.7424
Epoch 3854/4000
1545/1545 [==============================] - 0s 35us/step - loss: 0.6130 - acc: 0.7385
Epoch 3855/4000
1545/1545 [==============================] - 0s 34us/step - loss: 0.6130 - acc: 0.7346
Epoch 3856/4000
1545/1545 [==============================] - 0s 34us/step - loss: 0.6127 - acc: 0.7417
Epoch 3857/4000
1545/1545 [==============================] - 0s 38us/step - loss: 0.6118 - acc: 0.7417
Epoch 3858/4000
1545/1545 [==============================] - 0s 38us/step - loss: 0.6128 - acc: 0.7430
Epoch 3859/4000
1545/1545 [==============================] - 0s 41us/step - loss: 0.6108 - acc: 0.7424
Epoch 3860/4000
1545/1545 [==============================] - 0s 33us/step - loss: 0.6134 - acc: 0.7469
Epoch 3861/4000
1545/1545 [==============================] - 0s 33us/step - loss: 0.6132 - acc: 0.7430
Epoch 3862/4000
1545/1545 [==============================] - 0s 35us/step

Epoch 3933/4000
1545/1545 [==============================] - 0s 37us/step - loss: 0.6156 - acc: 0.7359
Epoch 3934/4000
1545/1545 [==============================] - 0s 40us/step - loss: 0.6119 - acc: 0.7430
Epoch 3935/4000
1545/1545 [==============================] - 0s 36us/step - loss: 0.6128 - acc: 0.7372
Epoch 3936/4000
1545/1545 [==============================] - 0s 32us/step - loss: 0.6114 - acc: 0.7392
Epoch 3937/4000
1545/1545 [==============================] - 0s 31us/step - loss: 0.6112 - acc: 0.7359
Epoch 3938/4000
1545/1545 [==============================] - 0s 28us/step - loss: 0.6102 - acc: 0.7437
Epoch 3939/4000
1545/1545 [==============================] - 0s 30us/step - loss: 0.6130 - acc: 0.7398
Epoch 3940/4000
1545/1545 [==============================] - 0s 29us/step - loss: 0.6115 - acc: 0.7411
Epoch 3941/4000
1545/1545 [==============================] - 0s 28us/step - loss: 0.6102 - acc: 0.7463
Epoch 3942/4000
1545/1545 [==============================] - 0s 31us/step

In [25]:
lossList=[]
for i in range(100):
    loss_value=model.evaluate([train_latent, train_PTcom, train_CARcom, train_PTasc, train_CARasc, train_SMasc], 
          train_y, batch_size=train_latent.shape[0],verbose=0)[0]
    lossList.append(loss_value)
print(np.mean(lossList))

0.6116320580244065


In [58]:
model.predict([train_latent[:1], train_PTcom[:1], train_CARcom[:1], train_PTasc[:1], train_CARasc[:1], train_SMasc[:1]])

array([[5.6687373e-01, 4.3312630e-01, 8.0049547e-13]], dtype=float32)

In [39]:
from tensorflow.keras.utils import plot_model
plot_model(idmodel, to_file='linearmodel.png',show_shapes=True)

# Debug part

In [41]:
i = 0
for layer in model.layers:
    print(layer.name, i)
    i += 1

input_latent 0
car_sys 1
carlover 2
input_PTasc 3
input_PTcom 4
input_CARasc 5
input_CARcom 6
input_SMasc 7
v0_as_2 8
util_common 9
v1_as_2 10
v2_as_2 11
V0 12
V1 13
V2 14
prediction 15


In [18]:
model.layers[9].get_weights()

[array([[-1.2084553]], dtype=float32), array([[-0.57047397]], dtype=float32)]

## LatentPart

In [55]:
for i in range(len(output_index)):
    print('{0:15s} {1}'.format(model.layers[output_index[i]].name, model.layers[output_index[i]].get_weights()))

car_sys         [array([[-0.12619334],
       [-0.09635082],
       [ 0.36885747],
       [-0.3496434 ],
       [ 0.6265547 ],
       [-0.06594128],
       [ 0.05845913],
       [ 0.6226478 ],
       [ 0.21824066],
       [ 0.09964096],
       [-0.04249339],
       [ 0.2796297 ],
       [-0.6184652 ]], dtype=float32), array([2.5612953], dtype=float32)]
carlover        []
v0_as_4         [array([[-2.0230172]], dtype=float32), array([[1.5660286]], dtype=float32), array([[-0.02796068]], dtype=float32)]
util_common     [array([[-1.172912]], dtype=float32), array([[-0.47131765]], dtype=float32)]
v1_as_4         [array([[5.2523074]], dtype=float32), array([[-6.434374]], dtype=float32)]
v2_as_4         [array([[-1.5794731]], dtype=float32), array([[-0.4133285]], dtype=float32), array([[1.9687463]], dtype=float32)]
V0              []
V1              []
V2              []
prediction      []


In [47]:
model.layers[9].get_weights()

[array([[-1.392449]], dtype=float32), array([[-0.55060613]], dtype=float32)]